In [2]:
import torch
from torch import nn

# 卷积
----------------
## 普通卷积

### 实现:

#### 官方实现
`torch.nn.Conv2d(in_chanels,out_channels,kernel_size,bias,stride,padding,padding_mode)`

**in_channels :**输入通道数目

**out_channels ：**输出通道数目

**kernel_size ：**卷积核大小，如果输入是一个值，比如 3 33，那么卷积核大小就是 3 × 3 3 \times 33×3 ，如果不想卷积核宽和高相等，还可以输入tuple类型数据，比如： ( 3 , 5 ) (3, 5)(3,5)

**stride ：**步长大小，跟上面卷积核参数一样，如果输入是一个值，比如 2 22 ，步长就是 2 × 2 2 \times 22×2 ，还可以输入元组 ( 2 , 1 ) (2, 1)(2,1) ，表示卷积核每次向右移动 1 11 个步长，向下移动 2 22 个步长。

**padding ：**填充，参数表示在周围补0的情况。补0的方向为上、下、左、右四个方向。如果是输入是单个值，比如1，就是在上下左右四个方向补一圈0。如果输入是元组比如 (2,1) ，表示在上下两个方向各补两行0，在左右两个方向各补一列0。

**bias ：**偏置，布尔类型，默认为 True ，即增加一个学习的偏置项。

**padding_mode ：**填充的模式，默认是 zero ，还可以选择 reflect 、 replicate 、 circular 。（有需要自行深入了解）
* valid padding（有效填充）：完全不使用填充。
* half/same padding（半填充/相同填充）：保证输入和输出的feature map尺寸相同。
* full padding（全填充）：在卷积操作过程中，每个像素在每个方向上被访问的次数相同。
* arbitrary padding（任意填充）：人为设定填充。

In [27]:
#手动实现
def conv2d(x, weight, bias, stride = 1 , pad = 0):      
    #参数x:输入; weight:权重; bias:偏重; stride:步长; pad:填充;
    n, c, h_in, w_in = x.shape
    d, c, k, j = weight.shape
    x_pad = torch.zeros(n, c, h_in+2*pad, w_in+2*pad)   # 对输入进行补零操作
    if pad>0:
        x_pad[:, :, pad:-pad, pad:-pad] = x
    else:
        x_pad = x

    x_pad = x_pad.unfold(2, k, stride)
    x_pad = x_pad.unfold(3, j, stride)           # 按照滑动窗展开
    out = torch.einsum(                          # 按照滑动窗相乘，
        'nchwkj,dckj->ndhw',                     # 并将所有输入通道卷积结果累加
        x_pad, weight)
    out = out + bias.view(1, -1, 1, 1)           # 添加偏置值
    return out

## 跨步卷积
跨步卷积主要改进的地方为，对滑动窗口的步幅进行调整，从原先的固定为1，转换为移动stride长度。

**跨步卷积去除了部分冗余且缩小了输出的尺寸，一定程度上取到了池化层的作用**

实现方式同上，但是将Stride设置不为1即可。

## 空洞卷积
空洞卷积中有一个额外的特殊参数扩张率对卷积核中相邻的数值进行填充

#### 官方实现
`torch.nn.Conv2d(in_chanels,out_channels,kernel_size,bias,dilation)`

**dilation：** 空洞率 

## 分组卷积

## 深度卷积